# BASELINE

In [116]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

% matplotlib inline

import seaborn as sns

pd.set_option('display.max_columns', 500)

from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.tree import ExtraTreeClassifier
from sklearn.decomposition import PCA

from sklearn.metrics import f1_score
from sklearn.manifold import TSNE

import lightgbm as lgb

In [117]:
mapping_target = {
    'A': 0,
    'A+': 1,
    'B': 2,
    'C': 3
}

ddd = {}
for k, v in mapping_target.items():
    ddd[float(v)] = k

In [118]:
df = pd.read_csv('prepared_data_v1.csv')

/Users/endnikita/projects/ml_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [119]:
df.head()

,Unnamed: 0,id,wb,razdel,source_type,offer_type,region,district,street,object,land_plot_area,common_area,offer_price,number_of_rooms,object_floor,object_number_of_floors,comment,creation_date,agency_name,agency_phone,price_rub_m_sq,longitude,latitude,object_type,upload_date,update_date2,relevance,geoimp,hide,adrmade,adrid,common_floor,erior_finish,level,plot_common_area,plzemkvmbig,common_price,common_price_m_sq,heating,building_type_fixed,build_year,ceiling_height,parking,sumy,summ,sumkvy,sumkvm,clusterid,segment,object_class_fixed,target
0,0,46054807,8,0,6,250,0,37,6031,6702,62.780636,411.0,1403518.0,1.0,25.0,58.0,26387,1.508630e+12,155,386,0.0,37.537371,55.746593,1,1.508630e+12,1.508630e+12,False,1,0,6407,476734.0,25.0,5,6,60.107315,302793.951939,1403518.0,75791.559618,7,81,1993.729818,12.310502,90,16842216.0,1403518.0,40978.63,3414.89,749.0,0,A,0
1,1,45591473,3,0,2,141,0,45,2558,738,62.780636,2025.0,6298620.0,1.0,2.0,3.0,4506,1.509235e+12,1105,386,0.0,37.641539,55.736539,1,1.507939e+12,1.509235e+12,True,1,0,2726,490041.0,2.0,5,6,60.107315,302793.951939,6298620.0,75791.559618,7,212,1993.729818,12.310502,90,75583440.0,6298620.0,37325.16,3110.43,749.0,0,A,0
2,2,45593704,3,0,2,141,0,116,1593,1961,62.780636,741.0,2470000.0,1.0,2.0,10.0,6239,1.509235e+12,1105,386,0.0,37.548088,55.752587,1,1.507939e+12,1.509235e+12,True,1,0,1479,472515.0,2.0,5,6,60.107315,302793.951939,2470000.0,75791.559618,7,212,1993.729818,12.310502,90,29640000.0,2470000.0,40000.00,3333.33,749.0,0,A,0
3,3,46362655,8,0,6,294,0,37,7323,7883,62.780636,2506.0,4934732.0,1.0,3.0,6.0,32221,1.508285e+12,1472,3078,0.0,37.624678,55.783944,1,1.508630e+12,1.508630e+12,False,1,0,7699,256122.0,3.0,5,6,60.107315,302793.951939,4934732.0,75791.559618,3,240,1993.729818,12.310502,90,59216784.0,4934732.0,23630.00,1969.17,749.0,0,A,0
4,4,46055844,8,0,6,304,0,37,7674,8209,62.780636,2200.0,3208333.0,1.0,2.0,10.0,39174,1.508285e+12,1472,3078,0.0,37.583562,55.789325,1,1.508630e+12,1.508630e+12,False,1,0,8050,2585839.0,2.0,5,5,60.107315,302793.951939,3208333.0,75791.559618,3,240,1993.729818,12.310502,90,38499996.0,3208333.0,17500.00,1458.33,749.0,0,B,2


In [120]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [121]:
df.head(10)

,id,wb,razdel,source_type,offer_type,region,district,street,object,land_plot_area,common_area,offer_price,number_of_rooms,object_floor,object_number_of_floors,comment,creation_date,agency_name,agency_phone,price_rub_m_sq,longitude,latitude,object_type,upload_date,update_date2,relevance,geoimp,hide,adrmade,adrid,common_floor,erior_finish,level,plot_common_area,plzemkvmbig,common_price,common_price_m_sq,heating,building_type_fixed,build_year,ceiling_height,parking,sumy,summ,sumkvy,sumkvm,clusterid,segment,object_class_fixed,target
0,46054807,8,0,6,250,0,37,6031,6702,62.780636,411.00,1403518.0,1.0,25.0,58.0,26387,1.508630e+12,155,386,0.0,37.537371,55.746593,1,1.508630e+12,1.508630e+12,False,1,0,6407,476734.0,25.0,5,6,60.107315,302793.951939,1.403518e+06,75791.559618,7,81,1993.729818,12.310502,90,16842216.0,1.403518e+06,40978.63,3414.890000,749.0,0,A,0
1,45591473,3,0,2,141,0,45,2558,738,62.780636,2025.00,6298620.0,1.0,2.0,3.0,4506,1.509235e+12,1105,386,0.0,37.641539,55.736539,1,1.507939e+12,1.509235e+12,True,1,0,2726,490041.0,2.0,5,6,60.107315,302793.951939,6.298620e+06,75791.559618,7,212,1993.729818,12.310502,90,75583440.0,6.298620e+06,37325.16,3110.430000,749.0,0,A,0
2,45593704,3,0,2,141,0,116,1593,1961,62.780636,741.00,2470000.0,1.0,2.0,10.0,6239,1.509235e+12,1105,386,0.0,37.548088,55.752587,1,1.507939e+12,1.509235e+12,True,1,0,1479,472515.0,2.0,5,6,60.107315,302793.951939,2.470000e+06,75791.559618,7,212,1993.729818,12.310502,90,29640000.0,2.470000e+06,40000.00,3333.330000,749.0,0,A,0
3,46362655,8,0,6,294,0,37,7323,7883,62.780636,2506.00,4934732.0,1.0,3.0,6.0,32221,1.508285e+12,1472,3078,0.0,37.624678,55.783944,1,1.508630e+12,1.508630e+12,False,1,0,7699,256122.0,3.0,5,6,60.107315,302793.951939,4.934732e+06,75791.559618,3,240,1993.729818,12.310502,90,59216784.0,4.934732e+06,23630.00,1969.170000,749.0,0,A,0
4,46055844,8,0,6,304,0,37,7674,8209,62.780636,2200.00,3208333.0,1.0,2.0,10.0,39174,1.508285e+12,1472,3078,0.0,37.583562,55.789325,1,1.508630e+12,1.508630e+12,False,1,0,8050,2585839.0,2.0,5,5,60.107315,302793.951939,3.208333e+06,75791.559618,3,240,1993.729818,12.310502,90,38499996.0,3.208333e+06,17500.00,1458.330000,749.0,0,B,2
5,3363173,7,1,5,587,0,37,47,10206,62.780636,99.92,15527568.0,1.0,4.0,4.0,23594,1.509235e+12,281,386,155400.0,37.525349,55.813074,1,1.475021e+12,1.508371e+12,True,1,0,144,1279234.0,4.0,5,6,60.107315,302793.951939,1.552757e+07,155400.000000,7,25,1993.729818,12.310502,147,15527568.0,2.631948e+06,155400.00,2779.551438,749.0,0,A,0
6,39682681,8,0,6,250,0,37,6037,6708,174.000000,270.00,0.0,1.0,49.0,60.0,10165,1.507162e+12,221,3933,39000.0,37.540829,55.748250,1,1.505002e+12,1.509235e+12,True,1,1,6413,255309.0,49.0,5,6,60.107315,17400.000000,2.845480e+08,39000.000000,1,100,1993.729818,12.310502,90,10530000.0,8.775000e+05,39000.00,3250.000000,749.0,0,A+,1
7,41741936,8,0,6,121,0,37,8556,3616,62.780636,890.00,0.0,1.0,1.0,2.0,21692,1.507507e+12,1509,477,24000.0,37.626502,55.770679,0,1.506211e+12,1.507853e+12,False,1,0,8932,1280995.0,1.0,5,4,60.107315,302793.951939,2.845480e+08,24000.000000,7,223,1993.729818,12.310502,90,21360000.0,1.780000e+06,24000.00,2000.000000,749.0,0,B,2
8,46056064,8,0,6,317,0,37,8062,8568,62.780636,1700.00,2960833.0,1.0,6.0,7.0,32258,1.508285e+12,1472,3078,0.0,37.634667,55.762201,1,1.508630e+12,1.508630e+12,False,1,0,8438,7110155.0,6.0,5,6,60.107315,302793.951939,2.960833e+06,75791.559618,3,223,1993.729818,12.310502,90,35529996.0,2.960833e+06,20900.00,1741.670000,749.0,0,B,2
9,45596599,3,0,2,141,1,37,10783,860,62.780636,2300.00,2204167.0,1.0,2.0,8.0,4429,1.509235e+12,1105,386,0.0,37.431612,55.882010,1,1.507939e+12,1.509235e+12,True,1,0,10384,4964317.0,2.0,5,6,60.107315,302793.951939,2.204167e+06,75791.559618,7,212,1993.729818,12.310502,90,26450004.0,2.204167e+06,11500.00,958.330000,1289.0,0,A,0


In [122]:
df_test = df[df.object_class_fixed.isnull()]

In [123]:
df_test.shape

(15639, 50)

In [124]:
df = df[df.object_class_fixed.notnull()]

In [125]:
df.shape

(36490, 50)

In [126]:
mapping_target = {
    'A': 0,
    'A+': 1,
    'B': 2,
    'C': 3
}

In [127]:
y = df['target']
df.drop(['target', 'id', 'object_class_fixed'], axis=1, inplace=True)

In [128]:
test_id = df_test['id']
df_test.drop(['target', 'id', 'object_class_fixed'], axis=1, inplace=True)

In [129]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=13)

In [72]:
# df_scaled = StandardScaler().fit_transform(df)
# X = pd.get_dummies(df)
# 
#         ('cat2', CatBoostClassifier(iterations=500, learning_rate=0.03, eval_metric='TotalF1', depth=10, loss_function='MultiClassOneVsAll', class_weights=[0.2, 0.023, 0.696, 0.078])),
#         ('lgb_gdbt', lgb.LGBMClassifier(num_leaves=31, learning_rate=0.01, max_depth=10, n_estimators=500, reg_alpha=0.2, reg_lambda=0.2)),
#         ('rf1', RandomForestClassifier(n_estimators=500, max_depth = 10, min_samples_leaf = 14, random_state = 42)),
#         ('lgb_goss', lgb.LGBMClassifier(num_leaves=31, learning_rate=0.01, max_depth=12, n_estimators=500, boosting_type='goss', reg_alpha=0.1, reg_lambda=0.15)),
#         ('lgb_dart', lgb.LGBMClassifier(num_leaves=31, learning_rate=0.01, max_depth=12, n_estimators=500, boosting_type='dart')),

# clf = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.01, max_depth=10, n_estimators=500, reg_alpha=0.2, reg_lambda=0.2)
# clf.fit(X_train, y_train)
# print(roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]))

In [37]:
cb_clf = CatBoostClassifier(iterations=500, learning_rate=0.1, eval_metric='TotalF1', depth=10, loss_function='MultiClass')
cb_clf.fit(X_train, y_train)    


cb_clf.predict(X_test)

print(f1_score(y_test, cb_clf.predict(X_test), average='macro'))

0:	learn: 0.7192604	total: 216ms	remaining: 1m 47s
1:	learn: 0.7281364	total: 614ms	remaining: 2m 32s
2:	learn: 0.7308483	total: 1.07s	remaining: 2m 57s
3:	learn: 0.7313269	total: 1.47s	remaining: 3m 1s
4:	learn: 0.7354972	total: 1.9s	remaining: 3m 8s
5:	learn: 0.7378730	total: 2.28s	remaining: 3m 7s
6:	learn: 0.7388584	total: 2.69s	remaining: 3m 9s
7:	learn: 0.7408255	total: 3.08s	remaining: 3m 9s
8:	learn: 0.7426339	total: 3.47s	remaining: 3m 9s
9:	learn: 0.7448895	total: 3.86s	remaining: 3m 9s
10:	learn: 0.7460615	total: 4.25s	remaining: 3m 8s
11:	learn: 0.7455306	total: 4.64s	remaining: 3m 8s
12:	learn: 0.7479177	total: 5.02s	remaining: 3m 8s
13:	learn: 0.7477345	total: 5.41s	remaining: 3m 7s
14:	learn: 0.7485406	total: 5.81s	remaining: 3m 7s
15:	learn: 0.7506967	total: 6.21s	remaining: 3m 7s
16:	learn: 0.7514840	total: 6.62s	remaining: 3m 8s
17:	learn: 0.7515314	total: 7.03s	remaining: 3m 8s
18:	learn: 0.7520875	total: 7.44s	remaining: 3m 8s
19:	learn: 0.7530578	total: 7.85s	remai

158:	learn: 0.8663657	total: 1m 3s	remaining: 2m 15s
159:	learn: 0.8665665	total: 1m 3s	remaining: 2m 15s
160:	learn: 0.8678954	total: 1m 4s	remaining: 2m 14s
161:	learn: 0.8685251	total: 1m 4s	remaining: 2m 14s
162:	learn: 0.8688339	total: 1m 4s	remaining: 2m 14s
163:	learn: 0.8690865	total: 1m 5s	remaining: 2m 13s
164:	learn: 0.8698550	total: 1m 5s	remaining: 2m 13s
165:	learn: 0.8703137	total: 1m 6s	remaining: 2m 12s
166:	learn: 0.8707062	total: 1m 6s	remaining: 2m 12s
167:	learn: 0.8715681	total: 1m 6s	remaining: 2m 12s
168:	learn: 0.8717438	total: 1m 7s	remaining: 2m 11s
169:	learn: 0.8720210	total: 1m 7s	remaining: 2m 11s
170:	learn: 0.8727153	total: 1m 8s	remaining: 2m 11s
171:	learn: 0.8730988	total: 1m 8s	remaining: 2m 10s
172:	learn: 0.8731077	total: 1m 8s	remaining: 2m 10s
173:	learn: 0.8736072	total: 1m 9s	remaining: 2m 9s
174:	learn: 0.8736179	total: 1m 9s	remaining: 2m 9s
175:	learn: 0.8747058	total: 1m 10s	remaining: 2m 9s
176:	learn: 0.8750024	total: 1m 10s	remaining: 2

312:	learn: 0.9183958	total: 2m 15s	remaining: 1m 20s
313:	learn: 0.9186246	total: 2m 15s	remaining: 1m 20s
314:	learn: 0.9184509	total: 2m 16s	remaining: 1m 19s
315:	learn: 0.9187808	total: 2m 16s	remaining: 1m 19s
316:	learn: 0.9193958	total: 2m 17s	remaining: 1m 19s
317:	learn: 0.9194123	total: 2m 17s	remaining: 1m 18s
318:	learn: 0.9194319	total: 2m 17s	remaining: 1m 18s
319:	learn: 0.9197392	total: 2m 18s	remaining: 1m 17s
320:	learn: 0.9203171	total: 2m 18s	remaining: 1m 17s
321:	learn: 0.9203455	total: 2m 19s	remaining: 1m 16s
322:	learn: 0.9206888	total: 2m 19s	remaining: 1m 16s
323:	learn: 0.9207387	total: 2m 20s	remaining: 1m 16s
324:	learn: 0.9211351	total: 2m 20s	remaining: 1m 15s
325:	learn: 0.9216003	total: 2m 21s	remaining: 1m 15s
326:	learn: 0.9219895	total: 2m 21s	remaining: 1m 14s
327:	learn: 0.9220150	total: 2m 22s	remaining: 1m 14s
328:	learn: 0.9219283	total: 2m 22s	remaining: 1m 14s
329:	learn: 0.9222496	total: 2m 22s	remaining: 1m 13s
330:	learn: 0.9229787	total:

468:	learn: 0.9507294	total: 3m 22s	remaining: 13.4s
469:	learn: 0.9508371	total: 3m 23s	remaining: 13s
470:	learn: 0.9511763	total: 3m 23s	remaining: 12.5s
471:	learn: 0.9511801	total: 3m 23s	remaining: 12.1s
472:	learn: 0.9513565	total: 3m 24s	remaining: 11.7s
473:	learn: 0.9512852	total: 3m 24s	remaining: 11.2s
474:	learn: 0.9512837	total: 3m 25s	remaining: 10.8s
475:	learn: 0.9516086	total: 3m 25s	remaining: 10.4s
476:	learn: 0.9517438	total: 3m 25s	remaining: 9.93s
477:	learn: 0.9517739	total: 3m 26s	remaining: 9.49s
478:	learn: 0.9519251	total: 3m 26s	remaining: 9.06s
479:	learn: 0.9520590	total: 3m 27s	remaining: 8.63s
480:	learn: 0.9523189	total: 3m 27s	remaining: 8.2s
481:	learn: 0.9524346	total: 3m 27s	remaining: 7.76s
482:	learn: 0.9524599	total: 3m 28s	remaining: 7.33s
483:	learn: 0.9526447	total: 3m 28s	remaining: 6.9s
484:	learn: 0.9528354	total: 3m 29s	remaining: 6.47s
485:	learn: 0.9529345	total: 3m 29s	remaining: 6.04s
486:	learn: 0.9530071	total: 3m 30s	remaining: 5.6

In [73]:
df_test['object_class_fixed'] = cb_clf.predict(df_test)
df_test['id'] = test_id

In [74]:
df_test['object_class_fixed'] = df_test.object_class_fixed.apply(lambda x:ddd[x])

In [75]:
df_test.to_csv('v1.csv')

In [76]:
df_test.head()

,wb,razdel,source_type,offer_type,region,district,street,object,land_plot_area,common_area,offer_price,number_of_rooms,object_floor,object_number_of_floors,comment,creation_date,agency_name,agency_phone,price_rub_m_sq,longitude,latitude,object_type,upload_date,update_date2,relevance,geoimp,hide,adrmade,adrid,common_floor,erior_finish,level,plot_common_area,plzemkvmbig,common_price,common_price_m_sq,heating,building_type_fixed,build_year,ceiling_height,parking,sumy,summ,sumkvy,sumkvm,clusterid,segment,object_class_fixed,id
36490,3,0,2,141,0,45,2148,217,62.780636,1173.0,3195056.0,1.0,2.0,2.0,7523,1.509062e+12,1105,386,0.0,37.635566,55.743411,1,1.507939e+12,1.509235e+12,True,1,0,2207,483661.0,2.0,5,6,60.107315,302793.951939,3.195056e+06,75791.559618,7,212,1993.729818,12.310502,90,38340672.0,3.195056e+06,32685.99,2723.830000,749.0,0,A,45410558
36491,16,1,6,577,3,37,10586,3081,5.000000,697.0,30600000.0,1.0,4.0,13.0,32814,1.508198e+12,1105,1592,0.0,60.621891,56.834062,1,1.507334e+12,1.508458e+12,True,1,0,10764,3894834.0,4.0,5,6,60.107315,5000.000000,3.060000e+07,75791.559618,7,81,1993.729818,3.000000,90,30600000.0,2.631948e+06,43902.44,2779.551438,331.0,0,B,43780787
36492,3,0,2,141,0,162,959,46,62.780636,100.0,100000.0,1.0,2.0,2.0,1388,1.509235e+12,1105,386,0.0,37.620501,55.627656,1,1.507939e+12,1.509235e+12,True,1,0,603,463984.0,2.0,5,6,60.107315,302793.951939,1.000000e+05,75791.559618,7,212,1993.729818,12.310502,90,1200000.0,1.000000e+05,12000.00,1000.000000,749.0,0,B,45415164
36493,8,0,6,142,0,37,5236,5977,62.780636,52.0,0.0,1.0,1.0,7.0,36371,1.508803e+12,1413,929,1347.0,37.673951,55.774634,1,1.508026e+12,1.509235e+12,True,1,0,5612,476229.0,1.0,5,4,60.107315,302793.951939,2.845480e+08,1347.000000,7,223,1993.729818,12.310502,90,840528.0,7.004400e+04,16164.00,1347.000000,749.0,0,B,45600788
36494,7,1,5,587,0,37,173,10206,62.780636,179.0,30609000.0,1.0,4.0,4.0,16907,1.509235e+12,281,386,171000.0,37.540637,55.649971,1,1.501027e+12,1.508371e+12,True,1,1,557,60449.0,4.0,5,6,60.107315,302793.951939,3.060900e+07,171000.000000,7,73,1993.729818,12.310502,396,30609000.0,2.631948e+06,171000.00,2779.551438,749.0,0,B,34024399


In [77]:
vclf = VotingClassifier(
    estimators=[
        ('cat1', CatBoostClassifier(n_estimators=1000, learning_rate=0.05, eval_metric='TotalF1', depth=5, loss_function='MultiClass')),
        ('cat3', CatBoostClassifier(
                         learning_rate=1.,
                         eval_metric='TotalF1',
                         loss_function='MultiClass',
                         depth=7,
                         n_estimators=500)),
        
    ], 
    voting='soft'
)

   

In [78]:
vclf.fit(X_train, y_train) 

0:	learn: 0.6972333	total: 108ms	remaining: 1m 47s
1:	learn: 0.7161761	total: 218ms	remaining: 1m 48s
2:	learn: 0.7202215	total: 328ms	remaining: 1m 48s
3:	learn: 0.7229233	total: 450ms	remaining: 1m 52s
4:	learn: 0.7254025	total: 562ms	remaining: 1m 51s
5:	learn: 0.7281919	total: 673ms	remaining: 1m 51s
6:	learn: 0.7269554	total: 785ms	remaining: 1m 51s
7:	learn: 0.7288336	total: 908ms	remaining: 1m 52s
8:	learn: 0.7292110	total: 1.03s	remaining: 1m 53s
9:	learn: 0.7294970	total: 1.14s	remaining: 1m 53s
10:	learn: 0.7297812	total: 1.25s	remaining: 1m 52s
11:	learn: 0.7298577	total: 1.37s	remaining: 1m 52s
12:	learn: 0.7263184	total: 1.48s	remaining: 1m 52s
13:	learn: 0.7269093	total: 1.6s	remaining: 1m 52s
14:	learn: 0.7271162	total: 1.71s	remaining: 1m 52s
15:	learn: 0.7284268	total: 1.83s	remaining: 1m 52s
16:	learn: 0.7281525	total: 1.94s	remaining: 1m 52s
17:	learn: 0.7290793	total: 2.06s	remaining: 1m 52s
18:	learn: 0.7290329	total: 2.17s	remaining: 1m 51s
19:	learn: 0.7290938	to

158:	learn: 0.7681380	total: 18.1s	remaining: 1m 35s
159:	learn: 0.7688733	total: 18.3s	remaining: 1m 35s
160:	learn: 0.7690948	total: 18.4s	remaining: 1m 35s
161:	learn: 0.7690832	total: 18.5s	remaining: 1m 35s
162:	learn: 0.7690610	total: 18.6s	remaining: 1m 35s
163:	learn: 0.7689784	total: 18.7s	remaining: 1m 35s
164:	learn: 0.7689847	total: 18.8s	remaining: 1m 35s
165:	learn: 0.7691835	total: 18.9s	remaining: 1m 35s
166:	learn: 0.7693046	total: 19s	remaining: 1m 34s
167:	learn: 0.7696747	total: 19.1s	remaining: 1m 34s
168:	learn: 0.7701743	total: 19.2s	remaining: 1m 34s
169:	learn: 0.7703010	total: 19.4s	remaining: 1m 34s
170:	learn: 0.7705755	total: 19.5s	remaining: 1m 34s
171:	learn: 0.7708107	total: 19.6s	remaining: 1m 34s
172:	learn: 0.7710584	total: 19.7s	remaining: 1m 34s
173:	learn: 0.7711581	total: 19.8s	remaining: 1m 34s
174:	learn: 0.7711771	total: 19.9s	remaining: 1m 33s
175:	learn: 0.7714493	total: 20s	remaining: 1m 33s
176:	learn: 0.7720652	total: 20.2s	remaining: 1m 3

314:	learn: 0.7949583	total: 35.9s	remaining: 1m 18s
315:	learn: 0.7949457	total: 36s	remaining: 1m 17s
316:	learn: 0.7950374	total: 36.1s	remaining: 1m 17s
317:	learn: 0.7951384	total: 36.2s	remaining: 1m 17s
318:	learn: 0.7952779	total: 36.3s	remaining: 1m 17s
319:	learn: 0.7955521	total: 36.5s	remaining: 1m 17s
320:	learn: 0.7956063	total: 36.6s	remaining: 1m 17s
321:	learn: 0.7956535	total: 36.7s	remaining: 1m 17s
322:	learn: 0.7955958	total: 36.8s	remaining: 1m 17s
323:	learn: 0.7961645	total: 36.9s	remaining: 1m 17s
324:	learn: 0.7962389	total: 37s	remaining: 1m 16s
325:	learn: 0.7963929	total: 37.1s	remaining: 1m 16s
326:	learn: 0.7964850	total: 37.2s	remaining: 1m 16s
327:	learn: 0.7966784	total: 37.4s	remaining: 1m 16s
328:	learn: 0.7966086	total: 37.5s	remaining: 1m 16s
329:	learn: 0.7967556	total: 37.6s	remaining: 1m 16s
330:	learn: 0.7971889	total: 37.7s	remaining: 1m 16s
331:	learn: 0.7972366	total: 37.8s	remaining: 1m 16s
332:	learn: 0.7973900	total: 37.9s	remaining: 1m 1

472:	learn: 0.8137628	total: 55.1s	remaining: 1m 1s
473:	learn: 0.8137896	total: 55.3s	remaining: 1m 1s
474:	learn: 0.8137689	total: 55.4s	remaining: 1m 1s
475:	learn: 0.8141781	total: 55.5s	remaining: 1m 1s
476:	learn: 0.8142361	total: 55.6s	remaining: 1m
477:	learn: 0.8145473	total: 55.7s	remaining: 1m
478:	learn: 0.8145073	total: 55.9s	remaining: 1m
479:	learn: 0.8145814	total: 56s	remaining: 1m
480:	learn: 0.8149382	total: 56.1s	remaining: 1m
481:	learn: 0.8150563	total: 56.2s	remaining: 1m
482:	learn: 0.8152198	total: 56.4s	remaining: 1m
483:	learn: 0.8151818	total: 56.5s	remaining: 1m
484:	learn: 0.8150420	total: 56.6s	remaining: 1m
485:	learn: 0.8158013	total: 56.7s	remaining: 60s
486:	learn: 0.8159613	total: 56.8s	remaining: 59.9s
487:	learn: 0.8161196	total: 57s	remaining: 59.8s
488:	learn: 0.8164959	total: 57.1s	remaining: 59.6s
489:	learn: 0.8165011	total: 57.2s	remaining: 59.5s
490:	learn: 0.8166924	total: 57.3s	remaining: 59.4s
491:	learn: 0.8171164	total: 57.4s	remaining:

632:	learn: 0.8337607	total: 1m 14s	remaining: 43.3s
633:	learn: 0.8339706	total: 1m 14s	remaining: 43.2s
634:	learn: 0.8339624	total: 1m 14s	remaining: 43.1s
635:	learn: 0.8340291	total: 1m 15s	remaining: 43s
636:	learn: 0.8342209	total: 1m 15s	remaining: 42.9s
637:	learn: 0.8343652	total: 1m 15s	remaining: 42.8s
638:	learn: 0.8343070	total: 1m 15s	remaining: 42.7s
639:	learn: 0.8345177	total: 1m 15s	remaining: 42.6s
640:	learn: 0.8345497	total: 1m 15s	remaining: 42.5s
641:	learn: 0.8347897	total: 1m 16s	remaining: 42.4s
642:	learn: 0.8345450	total: 1m 16s	remaining: 42.3s
643:	learn: 0.8347797	total: 1m 16s	remaining: 42.2s
644:	learn: 0.8349424	total: 1m 16s	remaining: 42.1s
645:	learn: 0.8351111	total: 1m 16s	remaining: 42s
646:	learn: 0.8351731	total: 1m 16s	remaining: 41.9s
647:	learn: 0.8353334	total: 1m 16s	remaining: 41.8s
648:	learn: 0.8353014	total: 1m 17s	remaining: 41.7s
649:	learn: 0.8355953	total: 1m 17s	remaining: 41.5s
650:	learn: 0.8356006	total: 1m 17s	remaining: 41.

...

944:	learn: 0.8550315	total: 1m 52s	remaining: 6.54s
945:	learn: 0.8548650	total: 1m 52s	remaining: 6.42s
946:	learn: 0.8548759	total: 1m 52s	remaining: 6.3s
947:	learn: 0.8550992	total: 1m 52s	remaining: 6.18s
948:	learn: 0.8550439	total: 1m 52s	remaining: 6.06s
949:	learn: 0.8550945	total: 1m 52s	remaining: 5.94s
950:	learn: 0.8551891	total: 1m 53s	remaining: 5.82s
951:	learn: 0.8553941	total: 1m 53s	remaining: 5.7s
952:	learn: 0.8554624	total: 1m 53s	remaining: 5.58s
953:	learn: 0.8556670	total: 1m 53s	remaining: 5.46s
954:	learn: 0.8555712	total: 1m 53s	remaining: 5.35s
955:	learn: 0.8554814	total: 1m 53s	remaining: 5.23s
956:	learn: 0.8553685	total: 1m 53s	remaining: 5.11s
957:	learn: 0.8558454	total: 1m 53s	remaining: 4.99s
958:	learn: 0.8558528	total: 1m 53s	remaining: 4.87s
959:	learn: 0.8558667	total: 1m 54s	remaining: 4.75s
960:	learn: 0.8559648	total: 1m 54s	remaining: 4.63s
961:	learn: 0.8561072	total: 1m 54s	remaining: 4.51s
962:	learn: 0.8561859	total: 1m 54s	remaining: 4

104:	learn: 0.9272201	total: 15.2s	remaining: 57.2s
105:	learn: 0.9280284	total: 15.3s	remaining: 57s
106:	learn: 0.9280196	total: 15.5s	remaining: 56.8s
107:	learn: 0.9287030	total: 15.6s	remaining: 56.6s
108:	learn: 0.9288526	total: 15.7s	remaining: 56.5s
109:	learn: 0.9295668	total: 15.9s	remaining: 56.3s
110:	learn: 0.9300240	total: 16s	remaining: 56.1s
111:	learn: 0.9301475	total: 16.1s	remaining: 55.9s
112:	learn: 0.9307924	total: 16.3s	remaining: 55.8s
113:	learn: 0.9312909	total: 16.4s	remaining: 55.6s
114:	learn: 0.9318895	total: 16.6s	remaining: 55.5s
115:	learn: 0.9326673	total: 16.7s	remaining: 55.3s
116:	learn: 0.9331759	total: 16.9s	remaining: 55.2s
117:	learn: 0.9342100	total: 17s	remaining: 55s
118:	learn: 0.9343539	total: 17.1s	remaining: 54.9s
119:	learn: 0.9347128	total: 17.3s	remaining: 54.7s
120:	learn: 0.9348414	total: 17.4s	remaining: 54.6s
121:	learn: 0.9350918	total: 17.6s	remaining: 54.4s
122:	learn: 0.9357132	total: 17.7s	remaining: 54.2s
123:	learn: 0.936596

263:	learn: 0.9828569	total: 40s	remaining: 35.8s
264:	learn: 0.9827822	total: 40.1s	remaining: 35.6s
265:	learn: 0.9830981	total: 40.3s	remaining: 35.4s
266:	learn: 0.9831278	total: 40.4s	remaining: 35.3s
267:	learn: 0.9833386	total: 40.6s	remaining: 35.1s
268:	learn: 0.9836488	total: 40.7s	remaining: 35s
269:	learn: 0.9839257	total: 40.9s	remaining: 34.8s
270:	learn: 0.9838583	total: 41s	remaining: 34.7s
271:	learn: 0.9840689	total: 41.2s	remaining: 34.5s
272:	learn: 0.9843085	total: 41.3s	remaining: 34.4s
273:	learn: 0.9846935	total: 41.5s	remaining: 34.2s
274:	learn: 0.9848327	total: 41.6s	remaining: 34s
275:	learn: 0.9849002	total: 41.8s	remaining: 33.9s
276:	learn: 0.9848341	total: 41.9s	remaining: 33.7s
277:	learn: 0.9848011	total: 42s	remaining: 33.6s
278:	learn: 0.9851817	total: 42.2s	remaining: 33.4s
279:	learn: 0.9853182	total: 42.3s	remaining: 33.2s
280:	learn: 0.9853545	total: 42.5s	remaining: 33.1s
281:	learn: 0.9854620	total: 42.6s	remaining: 32.9s
282:	learn: 0.9856666	

422:	learn: 0.9968102	total: 1m 4s	remaining: 11.8s
423:	learn: 0.9967757	total: 1m 5s	remaining: 11.7s
424:	learn: 0.9966728	total: 1m 5s	remaining: 11.5s
425:	learn: 0.9968789	total: 1m 5s	remaining: 11.3s
426:	learn: 0.9968790	total: 1m 5s	remaining: 11.2s
427:	learn: 0.9969134	total: 1m 5s	remaining: 11s
428:	learn: 0.9969821	total: 1m 5s	remaining: 10.9s
429:	learn: 0.9970850	total: 1m 5s	remaining: 10.7s
430:	learn: 0.9970161	total: 1m 6s	remaining: 10.6s
431:	learn: 0.9970847	total: 1m 6s	remaining: 10.4s
432:	learn: 0.9970851	total: 1m 6s	remaining: 10.3s
433:	learn: 0.9971536	total: 1m 6s	remaining: 10.1s
434:	learn: 0.9969811	total: 1m 6s	remaining: 9.96s
435:	learn: 0.9970845	total: 1m 6s	remaining: 9.8s
436:	learn: 0.9971533	total: 1m 6s	remaining: 9.65s
437:	learn: 0.9972564	total: 1m 7s	remaining: 9.5s
438:	learn: 0.9971879	total: 1m 7s	remaining: 9.35s
439:	learn: 0.9972222	total: 1m 7s	remaining: 9.19s
440:	learn: 0.9972565	total: 1m 7s	remaining: 9.04s
441:	learn: 0.99

VotingClassifier(estimators=[('cat1', <catboost.core.CatBoostClassifier object at 0x12281e518>), ('cat3', <catboost.core.CatBoostClassifier object at 0x12281eb00>)],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [95]:
df_test['object_class_fixed'] = vclf.predict(df_test)
df_test['id'] = test_id

df_test.to_csv('v2.csv')

/Users/endnikita/projects/ml_env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [130]:
df_top_test = pd.DataFrame()
df_top_test['id'] = test_id
df_top_test['object_class_fixed'] = vclf.predict(df_test)
df_top_test['object_class_fixed'] = df_top_test.object_class_fixed.apply(lambda x:ddd[x])

df_top_test[['id', 'object_class_fixed']].to_csv('v3.csv')

/Users/endnikita/projects/ml_env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [96]:
df_test['object_class_fixed'] = df_test.object_class_fixed.apply(lambda x:ddd[x])

In [16]:
print(f1_score(y_test, vclf.predict(X_test), average='macro'))

0.8002788126696248


/Users/endnikita/projects/ml_env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
rf1 = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.01, max_depth=10, n_estimators=500, reg_alpha=0.2, reg_lambda=0.2)
rf1.fit(X_train, y_train)
print(f1_score(y_test, rf1.predict(X_test), average='macro'))

In [ ]:
ct2 = CatBoostClassifier(iterations=500, learning_rate=0.03, eval_metric='TotalF1', depth=10, loss_function='MultiClassOneVsAll', class_weights=[0.2, 0.023, 0.696, 0.078])
ct2.fit(X_train, y_train)
print(f1_score(y_test, ct2.predict(X_test), average='macro'))

In [ ]:
ct3 = CatBoostClassifier(iterations=500, learning_rate=0.1, eval_metric='TotalF1', loss_function='MultiClass', depth=10, class_weights=[0.2, 0.023, 0.696, 0.078])
ct3.fit(X_train, y_train)
print(f1_score(y_test, ct2.predict(X_test), average='macro'))

In [ ]:
y.value_counts(normalize=True)

In [19]:
pipe = make_pipeline(
    StandardScaler(),
    GridSearchCV(
        LogisticRegression(class_weight={0:0.2, 1:0.023, 2:0.696, 3:0.078}, random_state=42),
        {
            'C': np.linspace(0.01, 1, 2, dtype=float),
        }
    )
)

scores = cross_val_score(estimator=pipe, X=df, y=y, cv=5, scoring='f1_macro')
print('KFold score: {0:.3f} ± {1:.3f}'.format(scores.mean(), scores.std()))

/Users/endnikita/projects/ml_env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/endnikita/projects/ml_env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/endnikita/projects/ml_env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


KFold score: 0.344 ± 0.007


/Users/endnikita/projects/ml_env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
# tree = RandomForestClassifier(n_estimators=500, max_depth = 10, min_samples_leaf = 14, random_state = 42)
# clf = CalibratedClassifierCV(tree, method='isotonic', cv=5)
# scores = cross_val_score(clf, df, y, cv=3, scoring='roc_auc')
# print(scores)

0        0
1        0
2        0
3        0
4        2
5        0
6        1
7        2
8        2
9        0
10       0
11       2
12       2
13       3
14       2
15       2
16       3
17       2
18       2
19       1
20       3
21       3
22       2
23       0
24       0
25       2
26       2
27       2
28       2
29       0
        ..
36460    0
36461    2
36462    2
36463    2
36464    2
36465    2
36466    2
36467    2
36468    0
36469    0
36470    2
36471    0
36472    2
36473    2
36474    2
36475    2
36476    3
36477    1
36478    2
36479    2
36480    3
36481    2
36482    0
36483    2
36484    0
36485    3
36486    2
36487    0
36488    0
36489    2
Name: target, Length: 36490, dtype: int64